## Exercise: Handle exceptions
With the knowledge of tracebacks and how exceptions are raised and handled, it's time to practice some of these concepts by using this notebook. We'll use a similar scenario where a program must read a configuration file called _config.txt_ and deal with errors as they show up.

In real-world scenarios, exceptions more clearly deal with separate functions. So this exercise will have different functions for different tasks regarding the configuration file.

> **TIP**
> Spend a few minutes trying to figure out a solution. Then scroll down to the bottom to see if you've managed to build the program according to specifications.

Start by creating a function which opens and reads the contents of the configuration file. Add a parameter to the function named `path` for the path to the configuration file.

In [ ]:
# Create a function that opens and reads the contents of the configuration file first
# Allow the function to accept a path. This will become handy later
def read_config(path):
    with open(path) as config_path:
        return config_path.readlines()

To mimic a brand-new environment, define a temporary directory that will act as the normal destination for the configuration file. You will use the `mkdtemp` method from `tempfile` library to create the directory. This will ensure we don't pollute the current notebook environment.

In [ ]:
import tempfile
import os 

# Create the temporary directory
config_dir = tempfile.mkdtemp()
config_path = os.path.join(config_dir, "config.txt")
print(config_dir)
print(config_path)

Now try to read the file using the `config_path` variable from before.

> **IMPORTANT** This will raise an error.

In [ ]:
# Now try reading the newly defined config.txt path
read_config(config_path)

That's a `FileNotFoundError` exception. In this case, we want to ensure that when there is no configuration file, a default configuration file is created with some basic values. Before you catch exceptions, create another function that writes the default values for the configuration file:

In [ ]:
import uuid

default_config_contents = f"""# Rocket Ship Configuration File!
fuel_tanks = 4
oxygen_tanks = 3
initial_propulsing_level = 84
unique_key = {uuid.uuid4()}"""

def write_default_config(path):
    with open(path, 'w') as default_config:
        default_config.write(default_config_contents)

Now you have a `read_config()` function that will get the contents of the configuration file. You also have a `write_default_config()` function that will write the default contents to a configuration file. You're ready to define the `main()` function that puts everything together. 

Write it up so that you can catch and handle the `FileNotFoundError` exception from before: 

In [ ]:
def main():
    try:
        config_contents = read_config(config_path)
    except FileNotFoundError:
        write_default_config(config_path)
        config_contents = read_config(config_path)
    # Output the contents to verify all worked fine
    print(config_contents)
    
main()
    

The `main()` function now handles `FileNotFoundError` and creates one by default with some contents. But so far, all you have is a list with contents. That's not very useful. 

Add a parser function that reads the values and creates a Python dictionary. Make the `main()` function show the dictionary after the parsing.

In [ ]:
def parse_config(contents):
    config = {}
    for line in contents:
        key, value = line.split('=')
        config[key] = value
    return config

In [ ]:
# Update the main() function to use the new parser
def main():
    try:
        config_contents = read_config(config_path)
    except FileNotFoundError:
        write_default_config(config_path)
        config_contents = read_config(config_path)
    # Output the contents to verify all worked fine
    print(parse_config(config_contents))
    
main()

Now there's a `ValueError` exception. This is happening in the parser function. You know that you can handle this exception, but where? There are several functions involved, and `main()` is currently handling exceptions.

Choosing where to handle exceptions isn't always straightforward. There's no particular rule in Python that helps here.

You can handle an exception anywhere in code. Use common sense and see what fits best. Remember that inadequate handling can always be improved later, as with everything in code. 

In this case, the parser itself should probably be responsible for handling the specifics of parsing a configuration file. That way, it can continue parsing the entire file while handling errors within the parsing.

In [ ]:
# Update the parser to handle ValueError
def parse_config(contents):
    config = {}
    for line in contents:
        try:
            key, value = line.split('=')
        except ValueError:
            # Ignore this error and skip to the next line
            continue
        config[key] = value
    return config
    
# Now call main() again and see if all is good now
main()

You're almost there. Somehow, the keys have trailing spaces, and some values have newline characters in them. Clean those up in the `parse_config()` function:

In [ ]:
def parse_config(contents):
    config = {}
    for line in contents:
        try:
            key, value = line.split('=')
        except ValueError:
            # Ignore this error and skip to the next line
            continue
        config[key.strip()] = value.strip('\n').strip()
    return config

main()

Good job! Now you have a few functions, each of which handles exceptions in its own way. Some functions delegate the error handling when doing the work (like the parsing one). Others, like the `main()` function, handle the logic at a higher level.